# Tutorial Completo: Métodos Monte Carlo en Reinforcement Learning## Objetivos de AprendizajeEn este notebook aprenderás:1. Los fundamentos matemáticos de los métodos Monte Carlo2. La diferencia entre First-Visit y Every-Visit MC3. MC Prediction para evaluación de políticas4. MC Control On-Policy con ε-greedy5. MC Control Off-Policy con Importance Sampling6. Aplicaciones prácticas: Blackjack, GridWorld, Cliff Walking**Duración estimada**: 3-4 horas**Prerequisitos**: Haber completado el notebook de Dynamic Programming

In [ ]:
import sysimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsimport pandas as pdfrom collections import defaultdictimport warningswarnings.filterwarnings("ignore")# Configuración de visualizaciónplt.style.use("seaborn-v0_8-darkgrid")sns.set_palette("husl")%matplotlib inline# Importar nuestras implementacionessys.path.append("/home/user/Reinforcement-learning-guide")from algoritmos_clasicos.monte_carlo.mc_prediction import (    MCPrediction, SimpleGridWorld, SimpleBlackjack)from algoritmos_clasicos.monte_carlo.mc_control import (    MCControlOnPolicy, MCControlOffPolicy, CliffWalking)print("✓ Importaciones exitosas")print(f"NumPy version: {np.__version__}")

## 1. Introducción Teórica### ¿Qué son los Métodos Monte Carlo?Los **métodos Monte Carlo (MC)** son una familia de algoritmos de aprendizaje por refuerzo que aprenden directamente de la experiencia sin necesidad de un modelo del entorno. A diferencia de la Programación Dinámica, MC aprende valores mediante el **promedio de retornos reales** observados en episodios completos.### Características Clave:1. **Model-Free**: No requieren conocer las probabilidades de transición $p(s',r|s,a)$2. **Basados en episodios**: Aprenden al finalizar episodios completos3. **Muestreo**: Usan experiencia real o simulada para estimar valores4. **Sin bootstrap**: No usan estimaciones de otros estados para actualizar5. **Unbiased estimates**: Las estimaciones convergen al valor verdadero### Diferencias con Dynamic Programming:| Aspecto | Dynamic Programming | Monte Carlo ||---------|-------------------|-------------|| **Modelo** | Requiere modelo completo | No requiere modelo (model-free) || **Actualización** | Bootstrapping (usa V(s')) | Usa retornos reales completos || **Convergencia** | Por sweep completo | Por episodio || **Aplicabilidad** | Solo entornos discretos con modelo | Cualquier entorno episódico || **Eficiencia** | Alta con modelo perfecto | Más eficiente en práctica sin modelo |

### Model-Free LearningMonte Carlo es el primer método verdaderamente **model-free** que estudiaremos:**¿Qué significa model-free?**- No necesitamos las dinámicas del entorno: $p(s',r|s,a)$- Aprendemos directamente de la interacción o simulación- Útil cuando el modelo es desconocido o demasiado complejo**Ejemplo intuitivo:**Imagina aprender a jugar Blackjack:- **DP**: Necesitarías calcular todas las probabilidades de cada carta- **MC**: Simplemente juegas muchas manos y promedias tus ganancias### Ventajas de Monte Carlo:1. **Flexibilidad**: Funciona con cualquier entorno simulable2. **Simplicidad conceptual**: "Intenta y promedia"3. **Foco en estados importantes**: Solo visita estados alcanzables4. **Bajo sesgo**: Estimaciones no sesgadas (unbiased)5. **Paralelizable**: Los episodios son independientes### Desventajas:1. **Alta varianza**: Los retornos pueden variar mucho2. **Solo episódico**: Requiere episodios que terminen3. **Lento para converger**: Necesita muchos episodios4. **Ineficiente para estados raros**: Requiere visitarlos repetidamente

### Aplicaciones Prácticas:- **Juegos**: Blackjack, Go, Poker (donde el modelo es complejo)- **Simulación de sistemas**: Cuando podemos simular pero no modelar analíticamente- **Robótica**: Aprender de episodios de interacción real- **Finanzas**: Valoración de opciones mediante simulación- **Planificación**: Problemas donde solo podemos samplear trayectorias

### Comparación Intuitiva: DP vs MC**Programación Dinámica (DP)**:```Para cada estado s:    V(s) = Σ p(s'|s,a)[r + γV(s')]  ← Usa MODELO           ↑    Necesita conocer todas las transiciones posibles```**Monte Carlo (MC)**:```Para cada estado s:    1. Genera episodios siguiendo política π    2. Observa returns reales G₁, G₂, G₃, ...    3. V(s) = promedio(G₁, G₂, G₃, ...)  ← Usa EXPERIENCIA```**Analogía del mundo real:**- **DP**: Estudiar un mapa completo antes de viajar- **MC**: Viajar muchas veces y aprender del camino

## 2. Fundamentos Matemáticos### 2.1 El Concepto de Return (Retorno)El **return** o **retorno** $G_t$ es la suma descontada de recompensas futuras desde el tiempo $t$:$$G_t = R_{t+1} + \gamma R_{t+2} + \gamma^2 R_{t+3} + \cdots = \sum_{k=0}^{\infty} \gamma^k R_{t+k+1}$$Donde:- $G_t$: Retorno desde el tiempo $t$- $R_{t+1}$: Recompensa inmediata- $\gamma \in [0,1]$: Factor de descuento- $k$: Pasos hacia el futuro**Propiedad recursiva:**$$G_t = R_{t+1} + \gamma G_{t+1}$$Esta propiedad es fundamental para el cálculo eficiente de retornos.### 2.2 Estimación de la Función de ValorLa función de valor de un estado bajo una política $\pi$ se define como:$$V^\pi(s) = \mathbb{E}_\pi[G_t \mid S_t = s]$$**Idea clave de Monte Carlo**: Estimar $V^\pi(s)$ mediante el **promedio empírico** de retornos observados:$$V(s) pprox rac{1}{N(s)} \sum_{i=1}^{N(s)} G_i(s)$$Donde:- $N(s)$: Número de veces que visitamos $s$- $G_i(s)$: Retorno observado en la i-ésima visita a $s$**Por la Ley de Grandes Números**: Cuando $N(s) 	o \infty$, $V(s) 	o V^\pi(s)$

### 2.3 First-Visit vs Every-Visit Monte CarloCuando un episodio visita el mismo estado múltiples veces, hay dos formas de procesar los retornos:#### First-Visit MC:Solo considera la **primera vez** que se visita un estado en cada episodio.$$V(s) = \text{promedio}\{G_t \mid S_t = s, \text{ y } t \text{ es la primera visita a } s \text{ en el episodio}\}$$**Propiedades:**- Cada episodio contribuye máximo una muestra por estado- Estimador insesgado (unbiased)- Garantiza convergencia a $V^\pi(s)$ cuando $n 	o \infty$#### Every-Visit MC:Considera **todas las visitas** a un estado en cada episodio.$$V(s) = \text{promedio}\{G_t \mid S_t = s\}$$**Propiedades:**- Cada episodio puede contribuir múltiples muestras por estado- También es insesgado asintóticamente- Puede converger más rápido en algunos casos- Más datos por episodio#### Comparación:| Aspecto | First-Visit | Every-Visit ||---------|------------|-------------|| **Muestras por episodio** | Una por estado | Múltiples posibles || **Sesgo** | Insesgado | Insesgado || **Varianza** | Mayor | Menor (más datos) || **Convergencia** | Garantizada | Garantizada || **Uso típico** | Más común teóricamente | Más común en práctica |

### 2.4 Ejemplo Numérico: Cálculo de ReturnsConsideremos un episodio simple con $\gamma = 0.9$:**Episodio**: $S_0 \xrightarrow{R_1=5} S_1 \xrightarrow{R_2=2} S_2 \xrightarrow{R_3=-1} S_3$ (terminal)**Cálculo de retornos (hacia atrás):**Para $t=2$ (último paso antes de terminal):$$G_2 = R_3 = -1$$Para $t=1$:$$G_1 = R_2 + \gamma G_2 = 2 + 0.9 \times (-1) = 2 - 0.9 = 1.1$$Para $t=0$ (inicio):$$G_0 = R_1 + \gamma G_1 = 5 + 0.9 \times 1.1 = 5 + 0.99 = 5.99$$**Interpretación:**- Desde $S_0$, el agente espera obtener un retorno total de 5.99- Desde $S_1$, el agente espera obtener 1.1- Desde $S_2$, el agente obtiene -1 (penalización final)Este cálculo se hace **después** de completar el episodio, no durante.

In [ ]:
# Ejemplo práctico: Calcular returns de un episodiodef calculate_returns(rewards, gamma=0.9):    """    Calcula los returns para cada paso del episodio.    Parámetros:    - rewards: lista de recompensas [R1, R2, R3, ...]    - gamma: factor de descuento    Retorna:    - returns: lista de returns [G0, G1, G2, ...]    """    returns = []    G = 0    # Procesar en reversa    for r in reversed(rewards):        G = r + gamma * G        returns.insert(0, G)    return returns# Ejemplo del episodio anteriorrewards = [5, 2, -1]gamma = 0.9returns = calculate_returns(rewards, gamma)print("Episodio: S0 → S1 → S2 → S3 (terminal)")print("="*50)print(f"\nRecompensas: {rewards}")print(f"Gamma: {gamma}\n")for t, (r, G) in enumerate(zip(rewards, returns)):    print(f"Paso t={t}:")    print(f"  Recompensa R_{t+1} = {r}")    print(f"  Return G_{t} = {G:.4f}")    print()# Verificar manualmenteprint("Verificación manual:")print(f"G_2 = R_3 = {rewards[2]}")print(f"G_1 = R_2 + γ*G_2 = {rewards[1]} + {gamma}*{rewards[2]} = {rewards[1] + gamma*rewards[2]}")print(f"G_0 = R_1 + γ*G_1 = {rewards[0]} + {gamma}*{returns[1]:.4f} = {rewards[0] + gamma*returns[1]:.4f}")

### 2.5 Actualización IncrementalEn la práctica, no almacenamos todos los retornos. Usamos una **actualización incremental** más eficiente:$$V(s) \leftarrow V(s) + \alpha [G - V(s)]$$Donde:- $\alpha$: Tasa de aprendizaje (learning rate)- $G$: Retorno observado- $V(s)$: Estimación actual- $[G - V(s)]$: Error de predicción**Equivalencia con promedio:**Si usamos $\alpha = \frac{1}{N(s)}$ (inverso del número de visitas), esto es equivalente al promedio:$$V_{n+1}(s) = V_n(s) + \frac{1}{n}[G_n - V_n(s)] = \frac{1}{n}\sum_{i=1}^{n} G_i$$**Ventaja de $\alpha$ constante:**Con $\alpha$ fijo (ej. $\alpha=0.1$), damos más peso a experiencias recientes, útil en entornos no estacionarios.

## 3. MC Prediction: Evaluación de Políticas### 3.1 Objetivo**MC Prediction** resuelve el problema de **evaluación de políticas**: dada una política $\pi$, estimar la función de valor $V^\pi(s)$.**Problema:** ¿Qué tan buena es una política dada?**Solución:** Generar muchos episodios siguiendo $\pi$ y promediar los retornos observados.### 3.2 Algoritmo First-Visit MC Prediction```Entrada:  - Política π a evaluar  - Número de episodios NInicializar:  - V(s) = 0 para todo s  - Returns(s) = lista vacía para todo sRepetir N veces:  1. Generar episodio siguiendo π:     S₀, A₀, R₁, S₁, A₁, R₂, ..., Sₜ  2. Para cada estado s que aparece en el episodio:     a. Calcular return G desde la primera vez que apareció s     b. Agregar G a Returns(s)     c. V(s) ← promedio(Returns(s))Retornar: V como estimación de V^π```### 3.3 Propiedades Teóricas**Teorema de Convergencia:**> First-Visit MC Prediction converge a $V^\pi(s)$ cuando el número de visitas a $s$ tiende a infinito:> $$\lim_{N(s) \to \infty} V(s) = V^\pi(s)$$**Tasa de convergencia:**El error estándar disminuye como $O(1/\sqrt{n})$ donde $n$ es el número de muestras.

### 3.4 Implementación: GridWorld con Política OptimistaVamos a evaluar una política simple en un GridWorld 5x5.

In [ ]:
# Crear entorno GridWorld 5x5print("="*70)print("MC PREDICTION - GridWorld 5x5")print("="*70)env_grid = SimpleGridWorld(size=5)print("\nDescripción del entorno:")print(f"  - Tamaño: 5x5 (25 estados)")print(f"  - Estado inicial: {env_grid.start}")print(f"  - Estado meta: {env_grid.goal}")print(f"  - Recompensas: +1.0 en meta, -0.01 por paso")print(f"  - Acciones: 0=↑, 1=→, 2=↓, 3=←")# Visualizar el entornoprint("\nEstructura del GridWorld:")print("┌─────┬─────┬─────┬─────┬─────┐")for i in range(5):    row = "│"    for j in range(5):        if (i, j) == env_grid.start:            row += "  S  │"        elif (i, j) == env_grid.goal:            row += "  G  │"        else:            row += "     │"    print(row)    if i < 4:        print("├─────┼─────┼─────┼─────┼─────┤")print("└─────┴─────┴─────┴─────┴─────┘")print("\nS = Start (inicio), G = Goal (meta)")

In [ ]:
# Definir una política "optimista" que tiende hacia la metadef create_biased_policy():    """    Política que prefiere moverse hacia abajo y derecha (hacia la meta).    70% hacia la meta, 30% aleatorio.    """    def policy(state):        row, col = state        # Con 70% de probabilidad, moverse hacia la meta        if np.random.random() < 0.7:            # Si no estamos en la última columna, ir a la derecha            if col < 4:                return 1  # derecha            # Si ya estamos en la última columna, ir abajo            elif row < 4:                return 2  # abajo        # 30% aleatorio        return np.random.randint(0, 4)    return policy# Crear políticapolicy_biased = create_biased_policy()# Probar la política con un episodioprint("\n" + "="*70)print("Probando la política optimista")print("="*70 + "\n")state = env_grid.reset()trajectory = [state]rewards = []for step in range(20):  # Máximo 20 pasos    action = policy_biased(state)    action_names = ['↑', '→', '↓', '←']    print(f"Paso {step}: Estado {state} → Acción {action_names[action]}", end="")    next_state, reward, done, _ = env_grid.step(action)    rewards.append(reward)    trajectory.append(next_state)    print(f" → Estado {next_state}, Recompensa {reward:.3f}")    if done:        print(f"\n¡Meta alcanzada en {step+1} pasos!")        break    state = next_stateprint(f"\nRetorno total del episodio: {sum(rewards):.4f}")print(f"Trayectoria: {' → '.join([str(s) for s in trajectory])}")

### 3.5 Entrenar MC Prediction: First-Visit

In [ ]:
# Crear evaluador MC Prediction (First-Visit)mc_first_visit = MCPrediction(gamma=0.99, method='first-visit')print("\n" + "="*70)print("ENTRENAMIENTO: First-Visit MC Prediction")print("="*70 + "\n")# Entrenar evaluando la políticaresults_first = mc_first_visit.evaluate_policy(    env=env_grid,    policy=policy_biased,    num_episodes=5000,    max_steps=100,    verbose=True)print("\n" + "="*70)print("Función de Valor Aprendida (muestra de estados)")print("="*70)# Mostrar valores de algunos estados clavestates_to_show = [    (0, 0), (0, 2), (0, 4),    (2, 0), (2, 2), (2, 4),    (4, 0), (4, 2), (4, 4)]for state in states_to_show:    if state in mc_first_visit.V:        visits = mc_first_visit.visit_counts[state]        value = mc_first_visit.V[state]        print(f"V{state} = {value:7.4f}  (visitas: {visits:5d})")    else:        print(f"V{state} = no visitado")

### 3.6 Visualización de la Función de Valor

In [ ]:
# Crear visualización de la función de valor como heatmapdef visualize_value_function_gridworld(V, visit_counts, size=5, title="Función de Valor"):    """    Visualiza la función de valor de un GridWorld como heatmap.    """    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))    # Crear matrices de valores y visitas    value_grid = np.zeros((size, size))    visits_grid = np.zeros((size, size))    for state, value in V.items():        row, col = state        value_grid[row, col] = value        visits_grid[row, col] = visit_counts.get(state, 0)    # Heatmap de valores    im1 = ax1.imshow(value_grid, cmap='RdYlGn', interpolation='nearest')    ax1.set_title(title, fontsize=14, fontweight='bold')    ax1.set_xlabel('Columna')    ax1.set_ylabel('Fila')    # Añadir valores numéricos    for i in range(size):        for j in range(size):            text = ax1.text(j, i, f'{value_grid[i, j]:.3f}',                          ha='center', va='center', color='black', fontsize=9)    # Marcar inicio y meta    ax1.plot(0, 0, 'go', markersize=15, label='Inicio')    ax1.plot(size-1, size-1, 'r*', markersize=20, label='Meta')    ax1.legend(loc='upper left')    plt.colorbar(im1, ax=ax1, label='Valor V(s)')    # Heatmap de visitas (logarítmico para mejor visualización)    visits_log = np.log10(visits_grid + 1)  # +1 para evitar log(0)    im2 = ax2.imshow(visits_log, cmap='Blues', interpolation='nearest')    ax2.set_title('Frecuencia de Visitas (log₁₀)', fontsize=14, fontweight='bold')    ax2.set_xlabel('Columna')    ax2.set_ylabel('Fila')    # Añadir números de visitas    for i in range(size):        for j in range(size):            text = ax2.text(j, i, f'{int(visits_grid[i, j])}',                          ha='center', va='center', color='white', fontsize=9,                          fontweight='bold')    plt.colorbar(im2, ax=ax2, label='log₁₀(visitas + 1)')    plt.tight_layout()    plt.show()# Visualizar resultadosvisualize_value_function_gridworld(    mc_first_visit.V,    mc_first_visit.visit_counts,    size=5,    title="Función de Valor - First-Visit MC")

### 3.7 Curvas de Aprendizaje

In [ ]:
# Visualizar curvas de aprendizajefig, axes = plt.subplots(2, 2, figsize=(15, 10))# 1. Returns por episodioax1 = axes[0, 0]ax1.plot(results_first['history']['mean_returns'], alpha=0.3, color='blue', label='Return crudo')# Suavizado con ventana móvilwindow = 100if len(results_first['history']['mean_returns']) >= window:    smoothed = pd.Series(results_first['history']['mean_returns']).rolling(window=window).mean()    ax1.plot(smoothed, linewidth=2, color='red', label=f'Media móvil ({window} eps)')ax1.set_xlabel('Episodio')ax1.set_ylabel('Return')ax1.set_title('Returns por Episodio', fontweight='bold')ax1.legend()ax1.grid(True, alpha=0.3)# 2. Cambios en valoresax2 = axes[0, 1]ax2.plot(results_first['history']['value_changes'], alpha=0.6, color='green')ax2.set_xlabel('Episodio')ax2.set_ylabel('Cambio máximo en V(s)')ax2.set_title('Cambios en la Función de Valor', fontweight='bold')ax2.set_yscale('log')ax2.grid(True, alpha=0.3)# 3. Estados visitados por episodioax3 = axes[1, 0]ax3.plot(results_first['history']['states_visited'], alpha=0.6, color='purple')ax3.set_xlabel('Episodio')ax3.set_ylabel('Estados únicos visitados')ax3.set_title('Exploración del Espacio de Estados', fontweight='bold')ax3.grid(True, alpha=0.3)# 4. Distribución de returnsax4 = axes[1, 1]ax4.hist(results_first['history']['mean_returns'], bins=50, alpha=0.7, color='orange', edgecolor='black')ax4.set_xlabel('Return')ax4.set_ylabel('Frecuencia')ax4.set_title('Distribución de Returns', fontweight='bold')ax4.axvline(np.mean(results_first['history']['mean_returns']),           color='red', linestyle='--', linewidth=2, label=f"Media: {np.mean(results_first['history']['mean_returns']):.3f}")ax4.legend()ax4.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.show()print(f"\nEstadísticas finales:")print(f"  Return promedio: {np.mean(results_first['history']['mean_returns']):.4f}")print(f"  Desviación estándar: {np.std(results_first['history']['mean_returns']):.4f}")print(f"  Return máximo: {np.max(results_first['history']['mean_returns']):.4f}")print(f"  Return mínimo: {np.min(results_first['history']['mean_returns']):.4f}")

### 3.8 Comparación: First-Visit vs Every-Visit MC

In [ ]:
# Entrenar Every-Visit MC para compararmc_every_visit = MCPrediction(gamma=0.99, method='every-visit')print("\n" + "="*70)print("ENTRENAMIENTO: Every-Visit MC Prediction")print("="*70 + "\n")results_every = mc_every_visit.evaluate_policy(    env=env_grid,    policy=policy_biased,    num_episodes=5000,    max_steps=100,    verbose=True)# Comparar First-Visit vs Every-Visitprint("\n" + "="*70)print("COMPARACIÓN: First-Visit vs Every-Visit")print("="*70 + "\n")print(f"{'Estado':<15} {'First-Visit':<15} {'Every-Visit':<15} {'Diferencia':<15}")print("-" * 65)for state in states_to_show:    if state in mc_first_visit.V and state in mc_every_visit.V:        v_first = mc_first_visit.V[state]        v_every = mc_every_visit.V[state]        diff = abs(v_first - v_every)        print(f"{str(state):<15} {v_first:<15.4f} {v_every:<15.4f} {diff:<15.6f}")print(f"\nDiferencia máxima: {max(abs(mc_first_visit.V.get(s, 0) - mc_every_visit.V.get(s, 0)) for s in mc_first_visit.V):.6f}")print("\n→ Ambos métodos convergen a valores muy similares")

### 3.9 Ejemplo: BlackjackAhora evaluaremos una política de Blackjack:- **Estado**: (suma_jugador, carta_visible_dealer, tiene_as_utilizable)- **Acciones**: 0=plantarse (stick), 1=pedir carta (hit)- **Política**: Pedir carta si suma < 20, plantarse si suma ≥ 20

In [ ]:
# Crear entorno Blackjack y políticaenv_blackjack = SimpleBlackjack()def blackjack_policy(state):    """Política conservadora: pedir si suma < 20"""    player_sum, dealer_showing, usable_ace = state    return 1 if player_sum < 20 else 0  # 1=hit, 0=stickprint("="*70)print("MC PREDICTION - Blackjack")print("="*70)# Evaluar política con MCmc_blackjack = MCPrediction(gamma=1.0, method='first-visit')  # gamma=1.0 para juegos finitosresults_blackjack = mc_blackjack.evaluate_policy(    env=env_blackjack,    policy=blackjack_policy,    num_episodes=10000,    max_steps=50,    verbose=True)# Mostrar algunos valores interesantesprint("\n" + "="*70)print("Función de Valor para estados seleccionados de Blackjack")print("="*70)print(f"\n{'Estado (suma, dealer, as)':<40} {'Valor V(s)':<15} {'Visitas':<10}")print("-" * 65)sample_states = [    (20, 10, False), (20, 7, False), (19, 10, False),    (19, 7, False), (18, 10, False), (18, 7, False),    (17, 10, False), (16, 7, False), (15, 10, False)]for state in sample_states:    if state in mc_blackjack.V:        value = mc_blackjack.V[state]        visits = mc_blackjack.visit_counts[state]        interpretation = "Favorable" if value > 0 else "Desfavorable"        print(f"{str(state):<40} {value:<15.4f} {visits:<10} ({interpretation})")

### Interpretación de Resultados Blackjack:- **Valores positivos**: El estado es favorable (alta probabilidad de ganar)- **Valores negativos**: El estado es desfavorable (alta probabilidad de perder)- **Cerca de 0**: Estado neutral (probabilidades equilibradas)**Observaciones típicas:**- Suma 20 contra cualquier carta del dealer: muy favorable (+0.6 a +0.8)- Suma 15-17 contra 10 del dealer: desfavorable (-0.3 a -0.5)- El as utilizable mejora las probabilidades en estados límite

## 4. MC Control On-Policy: Encontrando Políticas Óptimas### 4.1 De Prediction a ControlHasta ahora hemos usado MC Prediction para **evaluar** políticas dadas. Ahora queremos **encontrar** la política óptima.**MC Control** alterna entre dos pasos:1. **Policy Evaluation**: Estimar $Q^\pi(s,a)$ con MC Prediction2. **Policy Improvement**: Mejorar la política siendo greedy respecto a $Q$### 4.2 El Problema de ExploraciónPara aprender $Q(s,a)$, necesitamos visitar **todos** los pares (s,a).**Solución: Política ε-greedy**$$\pi(a|s) = \begin{cases}1 - \epsilon + \frac{\epsilon}{|A|} & \text{si } a = \arg\max_{a'} Q(s,a') \\\frac{\epsilon}{|A|} & \text{en otro caso}\end{cases}$$- Con probabilidad $\epsilon$: acción aleatoria (exploración)- Con probabilidad $1-\epsilon$: mejor acción conocida (explotación)### 4.3 Algoritmo MC Control On-Policy (ε-greedy)```Inicializar:  - Q(s,a) = 0 para todo s,a  - π = política ε-greedy respecto a Q  - Returns(s,a) = lista vacíaRepetir por cada episodio:  1. Generar episodio siguiendo π:     S₀, A₀, R₁, S₁, A₁, R₂, ..., Sₜ  2. Para cada par (s,a) en el episodio:     a. G ← return desde primera vez que apareció (s,a)     b. Agregar G a Returns(s,a)     c. Q(s,a) ← promedio(Returns(s,a))  3. Para cada s en el episodio:     π(s) ← ε-greedy respecto a Q(s,·)```

### 4.4 Implementación: GridWorld con ε-greedy

In [ ]:
# Crear agente MC Control On-Policyprint("="*70)print("MC CONTROL ON-POLICY - GridWorld 5x5")print("="*70)agent_on = MCControlOnPolicy(    gamma=0.99,    epsilon=0.2,      # 20% exploración inicial    epsilon_decay=0.999,  # Decaer epsilon gradualmente    epsilon_min=0.01,     # Mínimo 1% exploración    method='first-visit')# Crear nuevo entornoenv_control = SimpleGridWorld(size=5)print(f"\nParámetros del agente:")print(f"  - Gamma: {agent_on.gamma}")print(f"  - Epsilon inicial: {agent_on.epsilon}")print(f"  - Epsilon decay: {agent_on.epsilon_decay}")print(f"  - Epsilon mínimo: {agent_on.epsilon_min}")print(f"  - Método: {agent_on.method}")print(f"\nIniciando entrenamiento...")

In [ ]:
# Entrenar el agenteresults_on = agent_on.train(    env=env_control,    num_episodes=5000,    max_steps=100,    valid_actions=[0, 1, 2, 3],    verbose=True)print("\n" + "="*70)print("Entrenamiento completado!")print("="*70)

### 4.5 Visualización de la Política Aprendida

In [ ]:
# Extraer y visualizar política aprendidadef visualize_learned_policy(agent, size=5, title="Política Aprendida"):    """Visualiza la política y Q-values aprendidos"""    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))    action_symbols = ['↑', '→', '↓', '←']    # 1. Política determinista (greedy)    policy_grid = np.zeros((size, size), dtype=int)    for i in range(size):        for j in range(size):            state = (i, j)            if state in agent.Q and agent.Q[state]:                policy_grid[i, j] = max(agent.Q[state].items(), key=lambda x: x[1])[0]    ax1.set_title(title, fontsize=14, fontweight='bold')    ax1.set_xlim(-0.5, size - 0.5)    ax1.set_ylim(-0.5, size - 0.5)    ax1.set_xticks(range(size))    ax1.set_yticks(range(size))    ax1.grid(True, linewidth=2)    ax1.invert_yaxis()    for i in range(size):        for j in range(size):            action = policy_grid[i, j]            color = 'red' if (i, j) == (size-1, size-1) else 'blue'            ax1.text(j, i, action_symbols[action],                    ha='center', va='center', fontsize=24, color=color)    ax1.plot(0, 0, 'go', markersize=15, label='Inicio')    ax1.plot(size-1, size-1, 'r*', markersize=20, label='Meta')    ax1.legend(loc='upper left')    # 2. Valores Q máximos por estado    value_grid = np.zeros((size, size))    for i in range(size):        for j in range(size):            state = (i, j)            if state in agent.Q and agent.Q[state]:                value_grid[i, j] = max(agent.Q[state].values())    im = ax2.imshow(value_grid, cmap='RdYlGn', interpolation='nearest')    ax2.set_title('Valores Q Máximos V(s) = max_a Q(s,a)', fontsize=14, fontweight='bold')    ax2.set_xlabel('Columna')    ax2.set_ylabel('Fila')    for i in range(size):        for j in range(size):            ax2.text(j, i, f'{value_grid[i, j]:.2f}',                    ha='center', va='center', color='black', fontsize=9)    plt.colorbar(im, ax=ax2)    plt.tight_layout()    plt.show()# Visualizarvisualize_learned_policy(agent_on, size=5, title="Política Aprendida con MC Control On-Policy")

### 4.6 Análisis del Entrenamiento

In [ ]:
# Análisis de convergenciafig, axes = plt.subplots(2, 2, figsize=(15, 10))# 1. Returns por episodioax1 = axes[0, 0]ax1.plot(results_on['history']['episode_returns'], alpha=0.3, label='Return')window = 100smoothed = pd.Series(results_on['history']['episode_returns']).rolling(window=window).mean()ax1.plot(smoothed, linewidth=2, color='red', label=f'Media móvil ({window})')ax1.set_xlabel('Episodio')ax1.set_ylabel('Return')ax1.set_title('Evolución de Returns', fontweight='bold')ax1.legend()ax1.grid(True, alpha=0.3)# 2. Longitud de episodiosax2 = axes[0, 1]ax2.plot(results_on['history']['episode_lengths'], alpha=0.3, color='green', label='Longitud')smoothed_len = pd.Series(results_on['history']['episode_lengths']).rolling(window=window).mean()ax2.plot(smoothed_len, linewidth=2, color='darkgreen', label=f'Media móvil ({window})')ax2.set_xlabel('Episodio')ax2.set_ylabel('Longitud del episodio')ax2.set_title('Longitud de Episodios', fontweight='bold')ax2.legend()ax2.grid(True, alpha=0.3)# 3. Epsilon decayax3 = axes[1, 0]ax3.plot(results_on['history']['epsilon_history'], linewidth=2, color='orange')ax3.set_xlabel('Episodio')ax3.set_ylabel('Epsilon')ax3.set_title('Decaimiento de Epsilon (Exploración)', fontweight='bold')ax3.grid(True, alpha=0.3)# 4. Cambios en Q-valuesax4 = axes[1, 1]ax4.plot(results_on['history']['q_value_changes'], alpha=0.6, color='purple')ax4.set_xlabel('Episodio')ax4.set_ylabel('Cambio máximo en Q')ax4.set_title('Cambios en Q-Values', fontweight='bold')ax4.set_yscale('log')ax4.grid(True, alpha=0.3)plt.tight_layout()plt.show()# Estadísticas finalesprint("\nEstadísticas del Entrenamiento:")print("="*50)final_returns = results_on['history']['episode_returns'][-100:]final_lengths = results_on['history']['episode_lengths'][-100:]print(f"Últimos 100 episodios:")print(f"  Return promedio: {np.mean(final_returns):.4f} ± {np.std(final_returns):.4f}")print(f"  Longitud promedio: {np.mean(final_lengths):.2f} ± {np.std(final_lengths):.2f}")print(f"\nEpsilon final: {results_on['history']['epsilon_history'][-1]:.4f}")print(f"Estados-acción visitados: {sum(len(actions) for actions in agent_on.Q.values())}")

### 4.7 Evaluación de la Política Aprendida

In [ ]:
# Evaluar la política aprendida (100% greedy)print("\n" + "="*70)print("EVALUACIÓN: Política Aprendida (100% greedy, sin exploración)")print("="*70 + "\n")test_episodes = 100test_returns = []test_lengths = []test_trajectories = []for ep in range(test_episodes):    state = env_control.reset()    episode_return = 0    episode_length = 0    trajectory = [state]    for step in range(100):        # Usar política greedy (sin exploración)        action = agent_on.get_action(state, greedy=True)        state, reward, done, _ = env_control.step(action)        trajectory.append(state)        episode_return += reward        episode_length += 1        if done:            break    test_returns.append(episode_return)    test_lengths.append(episode_length)    if ep < 5:  # Guardar primeras 5 trayectorias        test_trajectories.append(trajectory)print(f"Resultados de evaluación ({test_episodes} episodios):")print(f"  Return promedio: {np.mean(test_returns):.4f} ± {np.std(test_returns):.4f}")print(f"  Longitud promedio: {np.mean(test_lengths):.2f} ± {np.std(test_lengths):.2f}")print(f"  Return máximo: {np.max(test_returns):.4f}")print(f"  Return mínimo: {np.min(test_returns):.4f}")print(f"\nPrimeras 5 trayectorias:")for i, traj in enumerate(test_trajectories, 1):    print(f"  {i}. {' → '.join([str(s) for s in traj])} (longitud: {len(traj)-1})")

## 5. MC Control Off-Policy: Importance Sampling### 5.1 Motivación**On-Policy**: Aprende sobre la política que está siguiendo (ε-greedy)- Ventaja: Convergencia estable- Desventaja: Nunca aprende la política óptima pura (siempre tiene exploración)**Off-Policy**: Aprende sobre la política target (óptima) mientras sigue una behavior policy (exploratoria)- **Behavior policy** $b(a|s)$: Política exploratoria que genera los episodios- **Target policy** $\pi(a|s)$: Política determinista que queremos aprender### 5.2 Importance SamplingEl problema: episodios generados por $b$ no siguen la distribución de $\pi$.**Solución**: Ponderar los retornos por el ratio de importancia:$$\rho_{t:T-1} = \prod_{k=t}^{T-1} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}$$Este ratio corrige el sesgo introducido por seguir $b$ en lugar de $\pi$.### 5.3 Weighted Importance SamplingPara reducir varianza, usamos **weighted importance sampling**:$$Q(s,a) = \frac{\sum_{i=1}^{n} \rho_i G_i}{\sum_{i=1}^{n} \rho_i}$$En lugar del promedio simple, ponderamos por los ratios de importancia.

### 5.4 Algoritmo MC Control Off-Policy```Inicializar:  - Q(s,a) = 0 para todo s,a  - C(s,a) = 0 (suma de pesos)  - π(s) = greedy respecto a Q  (target policy)  - b(s) = ε-greedy respecto a Q  (behavior policy)Repetir por cada episodio:  1. Generar episodio usando b:     S₀, A₀, R₁, ..., Sₜ  2. G ← 0  3. W ← 1  4. Para t = T-1, T-2, ..., 0:     a. G ← γG + R_{t+1}     b. C(Sₜ, Aₜ) ← C(Sₜ, Aₜ) + W     c. Q(Sₜ, Aₜ) ← Q(Sₜ, Aₜ) + (W/C(Sₜ, Aₜ))[G - Q(Sₜ, Aₜ)]     d. π(Sₜ) ← argmax_a Q(Sₜ, a)     e. Si Aₜ ≠ π(Sₜ): break  (episodio no sigue π)     f. W ← W · 1/b(Aₜ|Sₜ)```

### 5.5 Implementación: Cliff WalkingUsaremos el problema **Cliff Walking** donde:- Hay un "acantilado" que da recompensa -100- On-policy aprende camino **seguro** (lejos del acantilado)- Off-policy aprende camino **óptimo** (cerca del acantilado)

In [ ]:
# Crear entorno Cliff Walkingprint("="*70)print("MC CONTROL OFF-POLICY - Cliff Walking")print("="*70)env_cliff = CliffWalking()print(f"\nDescripción del entorno:")print(f"  - Tamaño: {env_cliff.rows}x{env_cliff.cols}")print(f"  - Inicio: {env_cliff.start}")print(f"  - Meta: {env_cliff.goal}")print(f"  - Acantilado: {len(env_cliff.cliff)} celdas")print(f"  - Caer al acantilado: -100, reinicia al inicio")print(f"  - Cada paso: -1")# Visualizarprint("\nMapa del Cliff Walking:")print("┌" + "─" * (env_cliff.cols * 4 + 1) + "┐")for i in range(env_cliff.rows):    row = "│"    for j in range(env_cliff.cols):        pos = (i, j)        if pos == env_cliff.start:            row += " S  "        elif pos == env_cliff.goal:            row += " G  "        elif pos in env_cliff.cliff:            row += " X  "        else:            row += "    "    row += "│"    print(row)print("└" + "─" * (env_cliff.cols * 4 + 1) + "┘")print("\nS=Start, G=Goal, X=Cliff (acantilado)")

In [ ]:
# Entrenar agente Off-Policyagent_off = MCControlOffPolicy(    gamma=0.99,    epsilon=0.3,  # Behavior policy más exploratoria    method='first-visit')print(f"\nParámetros del agente Off-Policy:")print(f"  - Gamma: {agent_off.gamma}")print(f"  - Epsilon (behavior): {agent_off.epsilon}")print(f"  - Target policy: greedy (determinista)")print(f"  - Behavior policy: ε-greedy (exploratoria)")print(f"\nIniciando entrenamiento Off-Policy...")results_off = agent_off.train(    env=env_cliff,    num_episodes=10000,  # Más episodios necesarios para off-policy    max_steps=200,    valid_actions=[0, 1, 2, 3],    verbose=True)

### 5.6 Comparación: On-Policy vs Off-Policy en Cliff Walking

In [ ]:
# Entrenar también On-Policy para compararprint("\n" + "="*70)print("Entrenando On-Policy para comparación...")print("="*70)agent_on_cliff = MCControlOnPolicy(    gamma=0.99,    epsilon=0.1,    epsilon_decay=0.999,    epsilon_min=0.01)results_on_cliff = agent_on_cliff.train(    env=env_cliff,    num_episodes=5000,    max_steps=200,    valid_actions=[0, 1, 2, 3],    verbose=False)print("\n✓ Ambos agentes entrenados")

In [ ]:
# Evaluar ambas políticasdef evaluate_cliff_policy(agent, env, episodes=100, is_on_policy=True):    """Evalúa una política en Cliff Walking"""    returns = []    lengths = []    falls = 0  # Contador de caídas al acantilado    for _ in range(episodes):        state = env.reset()        ep_return = 0        ep_length = 0        for step in range(500):            if is_on_policy and hasattr(agent, 'get_action'):                action = agent.get_action(state, greedy=True)            elif hasattr(agent, 'get_action'):                action = agent.get_action(state)            else:                action = 0            state, reward, done, _ = env.step(action)            ep_return += reward            ep_length += 1            if reward == -100:                falls += 1            if done:                break        returns.append(ep_return)        lengths.append(ep_length)    return {        'mean_return': np.mean(returns),        'std_return': np.std(returns),        'mean_length': np.mean(lengths),        'falls': falls    }# Evaluar ambosprint("\n" + "="*70)print("EVALUACIÓN EN CLIFF WALKING")print("="*70)eval_on = evaluate_cliff_policy(agent_on_cliff, env_cliff, episodes=200, is_on_policy=True)eval_off = evaluate_cliff_policy(agent_off, env_cliff, episodes=200, is_on_policy=False)print(f"\n{'Métrica':<25} {'On-Policy':<20} {'Off-Policy':<20}")print("-" * 65)print(f"{'Return promedio':<25} {eval_on['mean_return']:< 20.2f} {eval_off['mean_return']:< 20.2f}")print(f"{'Desv. estándar':<25} {eval_on['std_return']:< 20.2f} {eval_off['std_return']:< 20.2f}")print(f"{'Longitud promedio':<25} {eval_on['mean_length']:< 20.2f} {eval_off['mean_length']:< 20.2f}")print(f"{'Caídas al acantilado':<25} {eval_on['falls']:< 20} {eval_off['falls']:< 20}")print(f"\n{'Interpretación:'}")print(f"  - On-Policy: Aprende camino SEGURO (evita acantilado, más pasos)")print(f"  - Off-Policy: Aprende camino ÓPTIMO (cerca del acantilado, menos pasos)")print(f"  - Off-Policy obtiene mejor return pero es más arriesgado")

### 5.7 Visualización de Importance Ratios

In [ ]:
# Analizar importance ratiosfig, axes = plt.subplots(1, 2, figsize=(14, 6))# 1. Historia de importance ratiosax1 = axes[0]ratios = results_off['history']['importance_ratios']ax1.plot(ratios, alpha=0.5, color='purple')window = 100if len(ratios) >= window:    smoothed = pd.Series(ratios).rolling(window=window).mean()    ax1.plot(smoothed, linewidth=2, color='darkviolet', label=f'Media móvil ({window})')ax1.set_xlabel('Episodio')ax1.set_ylabel('Ratio de Importancia')ax1.set_title('Importance Ratios durante Entrenamiento', fontweight='bold')ax1.set_yscale('log')ax1.legend()ax1.grid(True, alpha=0.3)# 2. Distribución de ratiosax2 = axes[1]# Filtrar valores extremos para mejor visualizaciónfiltered_ratios = [r for r in ratios if r < np.percentile(ratios, 95)]ax2.hist(filtered_ratios, bins=50, alpha=0.7, color='purple', edgecolor='black')ax2.set_xlabel('Ratio de Importancia')ax2.set_ylabel('Frecuencia')ax2.set_title('Distribución de Importance Ratios', fontweight='bold')ax2.axvline(np.mean(filtered_ratios), color='red', linestyle='--',           linewidth=2, label=f'Media: {np.mean(filtered_ratios):.2f}')ax2.legend()ax2.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.show()print(f"\nEstadísticas de Importance Ratios:")print(f"  Media: {np.mean(ratios):.2f}")print(f"  Mediana: {np.median(ratios):.2f}")print(f"  Máximo: {np.max(ratios):.2f}")print(f"  % ratios > 10: {100 * np.mean([r > 10 for r in ratios]):.1f}%")print(f"\nRatios altos indican episodios con baja probabilidad bajo behavior policy")

## 6. Experimentos Completos y Análisis Comparativo### 6.1 Experimento 1: Efecto del Factor de Descuento γInvestigaremos cómo γ afecta el aprendizaje de MC.

In [ ]:
print("="*70)print("EXPERIMENTO 1: Efecto de Gamma")print("="*70)gammas = [0.7, 0.9, 0.99, 0.999]results_gamma = {}for gamma in gammas:    print(f"\nEntrenando con γ={gamma}...")    agent = MCControlOnPolicy(        gamma=gamma,        epsilon=0.2,        epsilon_decay=0.999,        epsilon_min=0.01    )    env_exp = SimpleGridWorld(size=5)    results = agent.train(        env=env_exp,        num_episodes=3000,        max_steps=100,        valid_actions=[0, 1, 2, 3],        verbose=False    )    # Evaluar    test_returns = []    for _ in range(50):        state = env_exp.reset()        ep_return = 0        for _ in range(100):            action = agent.get_action(state, greedy=True)            state, reward, done, _ = env_exp.step(action)            ep_return += reward            if done:                break        test_returns.append(ep_return)    results_gamma[gamma] = {        'agent': agent,        'mean_return': np.mean(test_returns),        'training_history': results['history']    }    print(f"  Return promedio: {np.mean(test_returns):.4f}")print("\n✓ Experimento completado")

In [ ]:
# Visualizar efectos de gammafig, axes = plt.subplots(1, 2, figsize=(14, 6))# 1. Returns por gammaax1 = axes[0]gamma_values = list(results_gamma.keys())mean_returns = [results_gamma[g]['mean_return'] for g in gamma_values]ax1.bar(range(len(gamma_values)), mean_returns, color='skyblue', edgecolor='black', alpha=0.7)ax1.set_xticks(range(len(gamma_values)))ax1.set_xticklabels([f'γ={g}' for g in gamma_values])ax1.set_ylabel('Return Promedio')ax1.set_title('Efecto de Gamma en Performance', fontweight='bold')ax1.grid(True, alpha=0.3, axis='y')# 2. Curvas de aprendizajeax2 = axes[1]for gamma in gammas:    history = results_gamma[gamma]['training_history']['episode_returns']    window = 100    smoothed = pd.Series(history).rolling(window=window).mean()    ax2.plot(smoothed, label=f'γ={gamma}', linewidth=2, alpha=0.7)ax2.set_xlabel('Episodio')ax2.set_ylabel('Return (suavizado)')ax2.set_title('Curvas de Aprendizaje por Gamma', fontweight='bold')ax2.legend()ax2.grid(True, alpha=0.3)plt.tight_layout()plt.show()print(f"\nObservaciones:")print(f"  - γ bajo (0.7): Aprende rápido pero políticas miopes")print(f"  - γ alto (0.99-0.999): Mejor performance pero más lento")print(f"  - Γ óptimo depende del horizonte del problema")

### 6.2 Experimento 2: Convergencia First-Visit vs Every-Visit

In [ ]:
print("\n" + "="*70)print("EXPERIMENTO 2: First-Visit vs Every-Visit Convergencia")print("="*70)# Entrenar ambos métodos múltiples vecesn_runs = 5episode_counts = [100, 500, 1000, 2000, 5000]results_comparison = {'first-visit': [], 'every-visit': []}for method in ['first-visit', 'every-visit']:    print(f"\nEntrenando {method}...")    for n_eps in episode_counts:        run_results = []        for run in range(n_runs):            mc = MCPrediction(gamma=0.99, method=method)            env_exp = SimpleGridWorld(size=5)            mc.evaluate_policy(                env=env_exp,                policy=create_biased_policy(),                num_episodes=n_eps,                max_steps=100,                verbose=False            )            # Calcular valor promedio            avg_value = np.mean(list(mc.V.values())) if mc.V else 0            run_results.append(avg_value)        results_comparison[method].append({            'episodes': n_eps,            'mean': np.mean(run_results),            'std': np.std(run_results)        })    print(f"  ✓ Completado {method}")print("\n✓ Experimento completado")

In [ ]:
# Visualizar convergenciafig, ax = plt.subplots(figsize=(10, 6))for method in ['first-visit', 'every-visit']:    data = results_comparison[method]    episodes = [d['episodes'] for d in data]    means = [d['mean'] for d in data]    stds = [d['std'] for d in data]    ax.errorbar(episodes, means, yerr=stds, marker='o', markersize=8,               linewidth=2, capsize=5, label=method.capitalize(), alpha=0.7)ax.set_xlabel('Número de Episodios', fontsize=12)ax.set_ylabel('Valor Promedio V(s)', fontsize=12)ax.set_title('Convergencia: First-Visit vs Every-Visit', fontsize=14, fontweight='bold')ax.set_xscale('log')ax.legend(fontsize=11)ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print(f"\nConclusión:")print(f"  - Ambos métodos convergen al mismo valor")print(f"  - Every-Visit puede tener menor varianza (más datos por episodio)")print(f"  - First-Visit tiene garantías teóricas más fuertes")

## 7. Visualizaciones Avanzadas### 7.1 Mapa de Calor de Q-Values por Acción

In [ ]:
# Visualizar Q-values para cada accióndef visualize_q_values_by_action(agent, size=5):    """Visualiza Q(s,a) para cada acción en GridWorld"""    fig, axes = plt.subplots(2, 2, figsize=(14, 12))    action_names = ['Arriba (↑)', 'Derecha (→)', 'Abajo (↓)', 'Izquierda (←)']    for action in range(4):        ax = axes[action // 2, action % 2]        # Crear grid de Q-values para esta acción        q_grid = np.zeros((size, size))        for i in range(size):            for j in range(size):                state = (i, j)                if state in agent.Q and action in agent.Q[state]:                    q_grid[i, j] = agent.Q[state][action]        # Visualizar        im = ax.imshow(q_grid, cmap='coolwarm', interpolation='nearest')        ax.set_title(f'Q-Values: {action_names[action]}', fontsize=12, fontweight='bold')        ax.set_xlabel('Columna')        ax.set_ylabel('Fila')        # Añadir valores        for i in range(size):            for j in range(size):                ax.text(j, i, f'{q_grid[i, j]:.2f}',                       ha='center', va='center', color='white', fontsize=9,                       fontweight='bold')        plt.colorbar(im, ax=ax)    plt.tight_layout()    plt.show()# Visualizar Q-values del agente on-policyprint("Q-Values por acción (Agente On-Policy):\n")visualize_q_values_by_action(agent_on, size=5)

### 7.2 Comparación Visual de Políticas

In [ ]:
# Comparar políticas aprendidasfig, axes = plt.subplots(1, 3, figsize=(18, 6))action_symbols = ['↑', '→', '↓', '←']# Función helper para extraer políticadef extract_policy_grid(agent, size=5):    policy_grid = np.zeros((size, size), dtype=int)    for i in range(size):        for j in range(size):            state = (i, j)            if state in agent.Q and agent.Q[state]:                policy_grid[i, j] = max(agent.Q[state].items(), key=lambda x: x[1])[0]    return policy_grid# 1. Política On-Policyax1 = axes[0]policy_on = extract_policy_grid(agent_on, 5)ax1.set_title('On-Policy (ε-greedy)', fontsize=14, fontweight='bold')ax1.set_xlim(-0.5, 4.5)ax1.set_ylim(-0.5, 4.5)ax1.grid(True, linewidth=2)ax1.invert_yaxis()for i in range(5):    for j in range(5):        ax1.text(j, i, action_symbols[policy_on[i, j]],                ha='center', va='center', fontsize=20, color='blue')# 2. Política Off-Policy (si existe)ax2 = axes[1]if hasattr(agent_off, 'Q') and len(agent_off.Q) > 0:    policy_off = extract_policy_grid(agent_off, 5)    ax2.set_title('Off-Policy (Importance Sampling)', fontsize=14, fontweight='bold')    ax2.set_xlim(-0.5, 4.5)    ax2.set_ylim(-0.5, 4.5)    ax2.grid(True, linewidth=2)    ax2.invert_yaxis()    for i in range(5):        for j in range(5):            ax2.text(j, i, action_symbols[policy_off[i, j]],                    ha='center', va='center', fontsize=20, color='green')else:    ax2.text(0.5, 0.5, 'No disponible\n(entrenado en Cliff Walking)',            ha='center', va='center', transform=ax2.transAxes)    ax2.set_title('Off-Policy', fontsize=14, fontweight='bold')# 3. Política óptima teórica (para GridWorld simple)ax3 = axes[2]policy_optimal = np.array([    [1, 1, 1, 1, 2],    [1, 1, 1, 1, 2],    [1, 1, 1, 1, 2],    [1, 1, 1, 1, 2],    [1, 1, 1, 1, 0]])ax3.set_title('Política Óptima Teórica', fontsize=14, fontweight='bold')ax3.set_xlim(-0.5, 4.5)ax3.set_ylim(-0.5, 4.5)ax3.grid(True, linewidth=2)ax3.invert_yaxis()for i in range(5):    for j in range(5):        ax3.text(j, i, action_symbols[policy_optimal[i, j]],                ha='center', va='center', fontsize=20, color='red')plt.tight_layout()plt.show()print("\nObservación: Las políticas aprendidas deberían ser similares a la óptima")

## 8. Ejercicios Prácticos### Ejercicio 1: Modificar Recompensas**Objetivo**: Entender cómo las recompensas afectan el aprendizaje de MC.**Tarea**:1. Crea un GridWorld 5x5 con recompensas modificadas:   - step_reward = -0.1 (en lugar de -0.01)   - goal_reward = 10.0 (en lugar de 1.0)2. Entrena un agente MC Control On-Policy3. Compara la política aprendida con la original**Preguntas**:- ¿Cómo afectan las recompensas negativas altas al comportamiento?- ¿El agente aprende rutas más directas?- ¿Cuántos episodios necesita para converger?

In [ ]:
# EJERCICIO 1: Tu código aquí# Pista: Necesitas crear un nuevo entorno modificando SimpleGridWorld# o crear recompensas personalizadas# Tu código aquí...

### Ejercicio 2: Análisis de Varianza**Objetivo**: Comparar la varianza de First-Visit y Every-Visit MC.**Tarea**:1. Implementa una función que calcule la varianza de los retornos2. Ejecuta 10 entrenamientos de cada método (First-Visit y Every-Visit)3. Compara las varianzas de las estimaciones de valor4. Grafica boxplots de las distribuciones**Pregunta**: ¿Qué método tiene menor varianza y por qué?

In [ ]:
# EJERCICIO 2: Tu código aquídef analyze_variance(method='first-visit', n_runs=10, n_episodes=1000):    """    Analiza varianza de un método MC.    Retorna:        - variance_per_state: dict con varianza por estado        - mean_variance: varianza promedio    """    # Tu código aquí...    pass# Ejecutar análisis# results_first = analyze_variance('first-visit')# results_every = analyze_variance('every-visit')# ... comparar y visualizar ...

### Ejercicio 3: Blackjack Óptimo**Objetivo**: Encontrar la política óptima para Blackjack.**Tarea**:1. Usa MC Control On-Policy para aprender la política óptima de Blackjack2. Entrena por 50,000 episodios3. Visualiza la política aprendida como una tabla:   - Filas: suma del jugador (12-21)   - Columnas: carta visible del dealer (1-10)   - Valores: Hit(1) o Stick(0)4. Compara con estrategias conocidas de Blackjack**Pregunta**: ¿En qué situaciones la política aprendida difiere de tu intuición?

In [ ]:
# EJERCICIO 3: Tu código aquí# Pista 1: Usa MCControlOnPolicy con SimpleBlackjack# Pista 2: Para visualizar, crea una matriz de acciones por estado# Tu código aquí...

### Ejercicio 4: Importance Sampling en Práctica**Objetivo**: Implementar y analizar importance sampling manually.**Tarea**:1. Implementa una función que calcule importance ratios manualmente2. Genera 100 episodios con una política aleatoria3. Calcula los retornos ponderados para una política target diferente4. Compara con los retornos directos**Pregunta**: ¿Cuándo los importance ratios son muy altos y por qué es problemático?

In [ ]:
# EJERCICIO 4: Tu código aquídef manual_importance_sampling(episodes, behavior_policy, target_policy, gamma=0.99):    """    Calcula valores usando importance sampling manualmente.    Parámetros:        - episodes: lista de episodios [(s,a,r), ...]        - behavior_policy: función que da probabilidad π_b(a|s)        - target_policy: función que da probabilidad π_t(a|s)        - gamma: factor de descuento    Retorna:        - V: diccionario de valores estimados        - ratios: lista de importance ratios    """    # Tu código aquí...    pass# Ejemplo de uso:# episodes = [...]  # generar episodios# V, ratios = manual_importance_sampling(episodes, ...)# ... analizar ratios ...

### Ejercicio 5: Experimento de Escalabilidad**Objetivo**: Investigar cómo MC escala con el tamaño del problema.**Tarea**:1. Entrena agentes MC Control en GridWorlds de diferentes tamaños: 3x3, 5x5, 7x7, 10x102. Mide para cada tamaño:   - Tiempo de entrenamiento   - Episodios hasta convergencia   - Calidad de la política final3. Grafica escalabilidad**Preguntas**:- ¿Cómo crece el tiempo de entrenamiento con el tamaño?- ¿MC escala mejor que Dynamic Programming?- ¿Qué tamaño es el límite práctico para MC?

In [ ]:
# EJERCICIO 5: Tu código aquíimport timedef scalability_experiment(sizes=[3, 5, 7, 10], n_episodes=5000):    """    Experimenta con diferentes tamaños de GridWorld.    Retorna:        - results: dict con métricas por tamaño    """    results = {}    for size in sizes:        print(f"\nEntrenando en grid {size}x{size}...")        # Tu código aquí:        # 1. Crear entorno de tamaño 'size'        # 2. Entrenar agente        # 3. Medir tiempo y episodios        # 4. Evaluar política        pass    return results# Ejecutar experimento# results = scalability_experiment()# ... visualizar resultados ...

## 9. Conclusiones y Próximos Pasos### 9.1 Resumen de Conceptos Clave**Métodos Monte Carlo**:1. **Model-free**: No requieren modelo del entorno2. **Basados en episodios**: Aprenden de experiencia completa3. **Unbiased**: Estimaciones convergen a valores verdaderos4. **Alta varianza**: Requieren muchos episodios**Variantes estudiadas**:- **MC Prediction**: Evaluar políticas dadas- **First-Visit vs Every-Visit**: Trade-off entre garantías teóricas y eficiencia- **MC Control On-Policy**: Aprender política óptima con exploración- **MC Control Off-Policy**: Aprender política óptima mientras se explora**Importance Sampling**:- Permite aprender de episodios generados por otra política- Weighted IS reduce varianza- Crítico para off-policy learning

### 9.2 Comparación: DP vs MC| Aspecto | Dynamic Programming | Monte Carlo ||---------|-------------------|-------------|| **Modelo** | Requiere p(s',r\|s,a) | No requiere (model-free) || **Computación** | Sweeps sobre todos los estados | Solo estados visitados || **Convergencia** | Garantizada en pocos sweeps | Requiere muchos episodios || **Aplicabilidad** | Solo con modelo completo | Simulación o experiencia real || **Bootstrapping** | Sí (usa V(s')) | No (usa retornos completos) || **Sesgo** | Depende de V inicial | Insesgado || **Varianza** | Baja | Alta || **Mejor para** | Problemas pequeños con modelo | Problemas sin modelo, grandes |**¿Cuándo usar cada uno?****Usar DP cuando**:- Tienes modelo completo y preciso- Espacio de estados pequeño/mediano- Necesitas solución exacta rápida**Usar MC cuando**:- No tienes modelo o es muy complejo- Puedes simular episodios- Solo te importan estados frecuentes- Problemas episódicos naturalmente

### 9.3 Limitaciones de Monte Carlo1. **Solo entornos episódicos**: MC requiere que los episodios terminen   - No funciona en tareas continuas   - Solución: Temporal Difference Learning (siguiente tema)2. **Alta varianza**: Los retornos pueden variar mucho   - Requiere muchos episodios para converger   - Solución: TD learning (menor varianza)3. **Aprendizaje lento**: Debe esperar al final del episodio   - No puede actualizar durante el episodio   - Solución: TD learning (actualizaciones incrementales)4. **Importance sampling puede ser inestable**: Ratios muy altos   - Especialmente problemático con episodios largos   - Soluciones: Truncated IS, Per-decision IS### 9.4 Próximos TemasPara superar las limitaciones de MC, estudiaremos:1. **Temporal Difference (TD) Learning**: Combina ventajas de DP y MC   - SARSA: On-policy TD control   - Q-Learning: Off-policy TD control   - Actualizaciones incrementales durante el episodio2. **N-step methods**: Intermedio entre MC y TD3. **Function Approximation**: Manejar espacios de estados grandes   - Representaciones compactas de V y Q   - Deep Q-Networks (DQN)4. **Policy Gradient Methods**: Optimización directa de políticas   - REINFORCE   - Actor-Critic### 9.5 Referencias y Recursos**Libros**:- Sutton & Barto (2018): *Reinforcement Learning: An Introduction* - Capítulos 5 y 6- Bertsekas (2019): *Reinforcement Learning and Optimal Control***Papers Clásicos**:- Watkins (1989): "Learning from Delayed Rewards" (Q-learning)- Sutton (1988): "Learning to Predict by the Method of Temporal Differences"**Recursos Online**:- David Silver's RL Course (Lecture 4 & 5)- OpenAI Spinning Up- DeepMind x UCL RL Course---**¡Felicidades!** Has completado el tutorial de Métodos Monte Carlo en Reinforcement Learning.**Próximos pasos sugeridos**:1. Completar los ejercicios prácticos2. Experimentar con diferentes entornos (Gym, Gymnasium)3. Implementar variantes avanzadas (Per-decision IS, MC Tree Search)4. Continuar con el notebook de Temporal Difference Learning